# 连接

## 普通连接
```python
def __init__(self, host='localhost', port=6379,
             db=0, password=None, socket_timeout=None,
             socket_connect_timeout=None,
             socket_keepalive=None, socket_keepalive_options=None,
             connection_pool=None, unix_socket_path=None,
             encoding='utf-8', encoding_errors='strict',
             charset=None, errors=None,
             decode_responses=False, retry_on_timeout=False,
             ssl=False, ssl_keyfile=None, ssl_certfile=None,
             ssl_cert_reqs='required', ssl_ca_certs=None,
             max_connections=None):
```

In [66]:
import redis

host = 'localhost'
port = 6379

r = redis.Redis(host=host, port=port, password='123456', decode_responses=True)

r.set('foo', 'bar')
print(r.get('foo'))

bar


## 连接池

避免每次建立、释放连接的开销

默认，每个Redis实例都会维护一个自己的连接池。

可以直接建立一个连接池，然后作为参数Redis，这样就可以实现多个Redis实例共享一个连接池。　

In [4]:
import redis

host = 'localhost'
port = 6379

pool = redis.ConnectionPool(host=host, port=port, decode_responses=True)

r = redis.Redis(connection_pool=pool)

r.set('foo', 'bar')
print(r.get('foo'))

# 操作

## String
### set(name, value, ex=None, px=None, nx=False, xx=False)

在Redis中设置值，默认，不存在则创建，存在则修改
参数：
 - ex，过期时间（秒）
 - px，过期时间（毫秒）
 - nx，如果设置为True，则只有name不存在时，当前set操作才执行
 - xx，如果设置为True，则只有name存在时，当前set操作才执行

设置一个k-v

In [67]:
r.set('name', 'bigberg')
print(r.get('name'))

bigberg


设置过期时间为2秒
2 秒后获取该值为None

In [68]:
import time

r.set('name', 'bigberg', ex=2)
print(r.get('name'))
time.sleep(2)
print(r.get('name'))

bigberg
None


### setnx(name, value)

设置值，只有name不存在时，执行设置操作（添加）
 
如果是r.setnx('name', 'Tom') 则name的值还是'bigberg'

In [69]:
r.setnx('name', 'Tom')

True

### setex(name, time, value)

In [28]:
r.setex('name', 2, 'Jack')

True

### psetex(name, time_ms, value)

参数：
- time_ms，过期时间（数字毫秒 或 timedelta对象）

In [74]:
r.psetex('name', 2000, 'Jack')

True

### mset(mapping)

批量设置值

In [75]:
print(r.mset({'a': 1, 'b': 2}))
print(r.get('a'))
print(r.get('b'))

True
1
2


### get(name)　

获取值

### mget(keys, *args)

批量获取


In [76]:
print(r.mget('a', 'b'))

['1', '2']


### getset(name, value)

设置新值并获取原来的值

In [77]:
print(r.getset('name', 'Tom'))
print(r.get('name'))

None
Tom


### getrange(key, start, end)

如果是中文字符，在utf-8编码下，一个中文字符占3个字节

In [78]:
r.set('address', '杭州')
print(r.getrange('address', 0, -1))
print(r.getrange('address', 0, 2))

杭州
杭


### setrange(name, offset, value)

修改字符串内容，从指定字符串索引开始向后替换（新值太长时，则向后添加）

参数：
- offset，字符串的索引，字节（一个汉字三个字节）
- value，要设置的值

In [79]:
r.setrange('address', 0, '苏')
print(r.get('address'))

苏州


In [80]:
print(r.get('name'))
r.setrange('name', 2, 'ol')
print(r.get('name'))

Tom
Tool


### setbit(name, offset, value)

对name对应值的二进制表示的位进行操作
  
```
# 对name对应值的二进制表示的位进行操作
  
# 参数：
    # name，redis的name
    # offset，位的索引（将值变换成二进制后再进行索引）
    # value，值只能是 1 或 0
  
# 注：如果在Redis中有一个对应： n1 = "foo"，
        那么字符串foo的二进制表示为：01100110 01101111 01101111
    所以，如果执行 setbit('n1', 7, 1)，则就会将第7位设置为1，
        那么最终二进制则变成 01100111 01101111 01101111，即："goo"
        
```

In [81]:
print(r.get('name'))
r.setbit('name', 5, 0)
print(r.get('name'))

Tool
Pool


### getbit(name, offset)

获取name对应的值的二进制表示中的某位的值 （0或1）

In [86]:
r.get('name')

'Pool'

In [82]:
print(r.getbit('name', 3))

1


### bitcount(key, start=None, end=None)

获取name对应的值的二进制表示中 1 的个数

参数：
-  key，Redis的name
- start，位起始位置
- end，位结束位置

In [83]:
print(r.bitcount('name', 0, 20))

18


### strlen(name)

返回name对应值的字节长度（一个汉字3个字节）

In [85]:
print(r.get('address'))
print(r.strlen('address'))

苏州
6


### incr(name, amount=1)

自增 name对应的值，当name不存在时，则创建name＝amount，否则，则自增。
  
参数：
- name,Redis的name
- amount,自增数（必须是整数）

In [87]:
for i in range(5):
    print(r.incr('login_user'))

1
2
3
4
5


### incrbyfloat(name, amount=1.0)

自增 name对应的值，当name不存在时，则创建name＝amount，否则，则自增。
  
参数：
- name,Redis的name
- amount,自增数（浮点型）

In [88]:
for i in range(5):
    print(r.incrbyfloat('num', 1.2))

1.2
2.4
3.6
4.8
6.0


### decr(name, amount=1)

自减 name对应的值，当name不存在时，则创建name＝amount，否则，则自减。
  
参数：
- name,Redis的name
- amount,自减数（整数）

In [89]:
for i in range(5):
    print(r.decr('login_user'))

4
3
2
1
0


### append(key, value)

在redis name对应的值后面追加内容
  
参数：
- key, redis的name
- value, 要追加的字符串

In [90]:
print(r.get('address'))
print(r.append('address', '西湖区'))
print(r.get('address'))

苏州
15
苏州西湖区


## Hash